In [7]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)
mp_draw = mp.solutions.drawing_utils

# Function to classify basic gestures
def classify_gesture(landmarks):
    """
    Classifies hand gestures: Thumbs Up, Thumbs Down, and Fist
    """
    try:
        # Normalize using wrist as reference
        wrist = landmarks[0]
        
        # Normalize coordinates
        normalized_landmarks = np.array([
            [(lm.x - wrist.x), (lm.y - wrist.y)] for lm in landmarks
        ])

        # Distance calculations
        thumb_index_dist = np.linalg.norm(normalized_landmarks[4] - normalized_landmarks[8])
        thumb_pinky_dist = np.linalg.norm(normalized_landmarks[4] - normalized_landmarks[20])

        # **Gesture Classification**
        # Fist → All fingers are close together
        if thumb_index_dist < 0.1 and thumb_pinky_dist < 0.2:
            return "Fist "

        # Thumbs Up → Thumb above MCP joint
        elif normalized_landmarks[4][1] < normalized_landmarks[2][1]:
            return "Thumbs Up "

        # Thumbs Down → Thumb below MCP joint
        elif normalized_landmarks[4][1] > normalized_landmarks[2][1]:
            return "Thumbs Down "

        else:
            return "Unknown"
    except Exception as e:
        print(f"Error in classification: {e}")
        return "Unknown"

# Real-time Webcam Feed
cap = cv2.VideoCapture(0)

# Set camera resolution
cap.set(3, 1280)   # Width
cap.set(4, 720)    # Height

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break

    # Flip the image for natural viewing
    frame = cv2.flip(frame, 1)

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract and classify landmarks
            gesture = classify_gesture(hand_landmarks.landmark)

            # Display detected gesture
            cv2.putText(frame, f'Gesture: {gesture}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp

# Initialize the MediaPipe Hand module
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

# OpenCV video capture from your phone camera
cap = cv2.VideoCapture(2)  # Use phone camera

# 🎥 Video Writer Configuration with H.264 codec for better compatibility
fps = 30  # Fixed FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_filename = "finger_counter_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264 codec for compatibility
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

# Hand tracking configuration
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)
        
        # Preserve BGR format for better color quality
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hands
        result = hands.process(rgb_frame)

        # Initialize finger counts
        left_count = 0
        right_count = 0

        if result.multi_hand_landmarks and result.multi_handedness:
            
            for idx, (hand_landmarks, handedness) in enumerate(zip(result.multi_hand_landmarks, result.multi_handedness)):

                # Draw landmarks
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract landmarks
                landmarks = []
                for lm in hand_landmarks.landmark:
                    h, w, _ = frame.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    landmarks.append((cx, cy))

                # Finger tips and base points
                tip_ids = [4, 8, 12, 16, 20]
                fingers = []

                # Improved thumb detection for mirrored frame
                label = handedness.classification[0].label
                if label == "Left":
                    if landmarks[tip_ids[0]][0] > landmarks[tip_ids[0] - 1][0]:
                        fingers.append(1)  # Open
                    else:
                        fingers.append(0)  # Closed
                else:
                    if landmarks[tip_ids[0]][0] < landmarks[tip_ids[0] - 1][0]:
                        fingers.append(1)  # Open
                    else:
                        fingers.append(0)  # Closed

                # Detecting 4 fingers
                for id in range(1, 5):
                    if landmarks[tip_ids[id]][1] < landmarks[tip_ids[id] - 2][1]:
                        fingers.append(1)  # Open
                    else:
                        fingers.append(0)  # Closed

                # Count fingers
                finger_count = fingers.count(1)

                # Assign count based on handedness
                if label == "Left":
                    left_count = finger_count
                else:
                    right_count = finger_count

        # Total fingers
        total_fingers = left_count + right_count

        # Display the finger count for both hands
        cv2.putText(frame, f'Left Hand: {left_count}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)  # Blue

        cv2.putText(frame, f'Right Hand: {right_count}', (10, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)  # Blue

        # Display the total finger count in red
        cv2.putText(frame, f'Total Fingers: {total_fingers}', (10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Red

        # Write the processed frame to the video
        out.write(frame)

        # Show the frame
        cv2.imshow("Both Hands Finger Counter", frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Video saved as '{output_filename}' with stable FPS and H.264 codec")


✅ Video saved as 'finger_counter_output.mp4' with stable FPS and H.264 codec


In [22]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)
mp_draw = mp.solutions.drawing_utils

# 🎥 Video Capture and Writer Configuration
cap = cv2.VideoCapture(2)  # Use phone camera

fps = 30  # Fixed FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_filename = "gesture_recognition_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264 codec for compatibility
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

# Function to classify basic gestures
def classify_gesture(landmarks):
    """
    Classifies hand gestures: Thumbs Up, Thumbs Down, and Fist
    """
    try:
        # Normalize using wrist as reference
        wrist = landmarks[0]

        # Normalize coordinates
        normalized_landmarks = np.array([
            [(lm.x - wrist.x), (lm.y - wrist.y)] for lm in landmarks
        ])

        # Distance calculations
        thumb_index_dist = np.linalg.norm(normalized_landmarks[4] - normalized_landmarks[8])
        thumb_pinky_dist = np.linalg.norm(normalized_landmarks[4] - normalized_landmarks[20])

        # **Gesture Classification**
        # Fist → All fingers are close together
        if thumb_index_dist < 0.1 and thumb_pinky_dist < 0.2:
            return "Fist"

        # Thumbs Up → Thumb above MCP joint
        elif normalized_landmarks[4][1] < normalized_landmarks[2][1]:
            return "Thumbs Up"

        # Thumbs Down → Thumb below MCP joint
        elif normalized_landmarks[4][1] > normalized_landmarks[2][1]:
            return "Thumbs Down"

        else:
            return "Unknown"
    except Exception as e:
        print(f"Error in classification: {e}")
        return "Unknown"

# Real-time Webcam Feed
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break

    # Flip the image for natural viewing
    frame = cv2.flip(frame, 1)

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract and classify landmarks
            gesture = classify_gesture(hand_landmarks.landmark)

            # Display detected gesture
            cv2.putText(frame, f'Gesture: {gesture}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Write the processed frame to the video
    out.write(frame)

    # Show the frame
    cv2.imshow("Gesture Recognition", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Video saved as '{output_filename}' with gesture recognition")


✅ Video saved as 'gesture_recognition_output.mp4' with gesture recognition
